<a href="https://colab.research.google.com/github/rocketwolf98/ColabCompilation/blob/main/compstat_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install fireducks
# !pip install "xarray[complete]"
# !pip install pygrib
import fireducks.pandas as pd
from google.colab import drive
import cfgrib
import numpy as np

In [2]:
path = '/content/drive/MyDrive/Compstat/e8984ac9609d8ca6fb199e5787179e40.grib'

In [3]:
with cfgrib.open_dataset(path) as ds:
  df= ds.to_dataframe()

/usr/local/lib/python3.11/dist-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


In [ ]:
#export to parquet
df.to_parquet('data.parquet')

### Quick Analysis and Wrangling

In [6]:
small = df.head(5)

In [7]:
small.shape

(5, 7)

In [8]:
small.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5 entries, (Timestamp('2022-01-01 00:00:00'), 21.0, 116.0) to (Timestamp('2022-01-01 00:00:00'), 21.0, 117.0)
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   number      5 non-null      int64          
 1   step        5 non-null      timedelta64[ns]
 2   surface     5 non-null      float64        
 3   valid_time  5 non-null      datetime64[ns] 
 4   d2m         5 non-null      float32        
 5   t2m         5 non-null      float32        
 6   sp          5 non-null      float32        
dtypes: datetime64[ns](1), float32(3), float64(1), int64(1), timedelta64[ns](1)
memory usage: 1.2 MB


In [12]:
df.columns

Index(['number', 'step', 'surface', 'valid_time', 'd2m', 't2m', 'sp'], dtype='object')

In [9]:
small['t2m_C'] = [item - 273.15 for item in small['t2m']]

<ipython-input-9-d37a63ab9d5e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small['t2m_C'] = [item - 273.15 for item in small['t2m']]


In [11]:
small['d2m_C'] = [item -273.15 for item in small['d2m']]

<ipython-input-11-d0a92caa02a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small['d2m_C'] = [item -273.15 for item in small['d2m']]


In [12]:
small

number   step  surface valid_time         d2m  \
time       latitude longitude                                                  
2022-01-01 21.0     116.00          0 0 days      0.0 2022-01-01  289.669922   
                    116.25          0 0 days      0.0 2022-01-01  289.689453   
                    116.50          0 0 days      0.0 2022-01-01  289.839844   
                    116.75          0 0 days      0.0 2022-01-01  290.023438   
                    117.00          0 0 days      0.0 2022-01-01  290.099609   

                                      t2m           sp      t2m_C      d2m_C  
time       latitude longitude                                                 
2022-01-01 21.0     116.00     294.031494  102353.0625  20.881494  16.519922  
                    116.25     294.064697  102340.0625  20.914697  16.539453  
                    116.50     294.166260  102362.0625  21.016260  16.689844  
                    116.75     294.236572  102346.0625  21.086572  16.873438  
                    117.00     294.424072  102330.0625  21.274072  16.949609

In [15]:
def svp(T):
  return 0.61078 * np.exp((17.27 * T)/(T + 237.3))

def rel_humidity(T, Td):
  es_t = svp(T)
  es_Td = svp(Td)
  RH = 100 * (es_Td/es_t)
  return RH

small['rel_humidity'] = [rel_humidity(T2m, D2m) for T2m,D2m in zip(small['t2m'], small['d2m'])]

<ipython-input-15-50fc2181f1aa>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small['rel_humidity'] = [rel_humidity(T2m, D2m) for T2m,D2m in zip(small['t2m'], small['d2m'])]


In [16]:
small

number   step  surface valid_time         d2m  \
time       latitude longitude                                                  
2022-01-01 21.0     116.00          0 0 days      0.0 2022-01-01  289.669922   
                    116.25          0 0 days      0.0 2022-01-01  289.689453   
                    116.50          0 0 days      0.0 2022-01-01  289.839844   
                    116.75          0 0 days      0.0 2022-01-01  290.023438   
                    117.00          0 0 days      0.0 2022-01-01  290.099609   

                                      t2m           sp      t2m_C      d2m_C  \
time       latitude longitude                                                  
2022-01-01 21.0     116.00     294.031494  102353.0625  20.881494  16.519922   
                    116.25     294.064697  102340.0625  20.914697  16.539453   
                    116.50     294.166260  102362.0625  21.016260  16.689844   
                    116.75     294.236572  102346.0625  21.086572  16.873438   
                    117.00     294.424072  102330.0625  21.274072  16.949609   

                               rel_humidity  
time       latitude longitude                
2022-01-01 21.0     116.00        93.815663  
                    116.25        93.797489  
                    116.50        93.867372  
                    116.75        94.025832  
                    117.00        93.875857

### Feature Engineering

In [4]:
data = df[['d2m','t2m','sp','valid_time']]

In [5]:
#convert units to celsius
data['t2m_C'] = data['t2m'] - 273.15
data['d2m_C'] = data['d2m'] - 273.15

<ipython-input-5-540ddd2c69e5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['t2m_C'] = data['t2m'] - 273.15
<ipython-input-5-540ddd2c69e5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['d2m_C'] = data['d2m'] - 273.15


In [6]:
data

d2m         t2m           sp  \
time                latitude longitude                                        
2022-01-01 00:00:00 21.0     116.00     289.669922  294.031494  102353.0625   
                             116.25     289.689453  294.064697  102340.0625   
                             116.50     289.839844  294.166260  102362.0625   
                             116.75     290.023438  294.236572  102346.0625   
                             117.00     290.099609  294.424072  102330.0625   
...                                            ...         ...          ...   
2024-12-31 23:00:00 5.0      126.00     296.891846  300.148926  100994.6875   
                             126.25     296.919189  300.279785  101023.6875   
                             126.50     296.975830  300.074707  101018.6875   
                             126.75     297.010986  300.080566  101064.6875   
                             127.00     296.952393  300.078613  101069.6875   

                                                valid_time      t2m_C  \
time                latitude longitude                                  
2022-01-01 00:00:00 21.0     116.00    2022-01-01 00:00:00  20.881494   
                             116.25    2022-01-01 00:00:00  20.914697   
                             116.50    2022-01-01 00:00:00  21.016260   
                             116.75    2022-01-01 00:00:00  21.086572   
                             117.00    2022-01-01 00:00:00  21.274072   
...                                                    ...        ...   
2024-12-31 23:00:00 5.0      126.00    2024-12-31 23:00:00  26.998926   
                             126.25    2024-12-31 23:00:00  27.129785   
                             126.50    2024-12-31 23:00:00  26.924707   
                             126.75    2024-12-31 23:00:00  26.930566   
                             127.00    2024-12-31 23:00:00  26.928613   

                                            d2m_C  
time                latitude longitude             
2022-01-01 00:00:00 21.0     116.00     16.519922  
                             116.25     16.539453  
                             116.50     16.689844  
                             116.75     16.873438  
                             117.00     16.949609  
...                                           ...  
2024-12-31 23:00:00 5.0      126.00     23.741846  
                             126.25     23.769189  
                             126.50     23.825830  
                             126.75     23.860986  
                             127.00     23.802393  

[76939200 rows x 6 columns]

In [ ]:
data['rel_humidity'] = data.apply(lambda row: rel_humidity(row['t2m'], row['d2m']), axis=1)
